<a href="https://colab.research.google.com/github/CasualMathEnjoyer/01RAD_Ex09_HW/blob/main/ex09_hw_log_transform%2Bscaler_pro_kappa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD  Exercise 09 - Homework

Authors: Your names here  
Date: 2024-12-03  

---

## Task Description

The dataset is based on the **House Sales in King County, USA** dataset, which can be found, for example, on kaggle.com or in the `moderndive` library under the name `house_prices`. The original dataset contains house sale prices in the King County area, which includes Seattle, and the data was collected between May 2014 and May 2015. For our purposes, several variables have been removed, and the dataset has been significantly reduced and slightly modified.

The dataset has already been split into three parts and modified, all of which will be used progressively throughout this assignment.

---

## Variables Description

The dataset contains the following 18 variables, and our goal is to explore the influence of 12 of them on the target variable `price`.

| Feature         | Description                                           |
|------------------|-------------------------------------------------------|
| `id`            | Unique identifier for a house                         |
| `price`         | Sale price (prediction target)                        |
| `bedrooms`      | Number of bedrooms                                    |
| `bathrooms`     | Number of bathrooms                                   |
| `sqft_living`   | Square footage of the home                            |
| `sqft_lot`      | Square footage of the lot                             |
| `floors`        | Total number of floors (levels) in the house          |
| `waterfront`    | Whether the house has a waterfront view               |
| `view`          | Number of times the house has been viewed             |
| `condition`     | Overall condition of the house                        |
| `grade`         | Overall grade given to the housing unit               |
| `sqft_above`    | Square footage of the house apart from the basement   |
| `sqft_basement` | Square footage of the basement                        |
| `yr_built`      | Year the house was built                              |
| `yr_renovated`  | Year when the house was renovated                     |
| `sqft_living15` | Living room area in 2015 (after renovations)          |
| `sqft_lot15`    | Lot size in 2015 (after renovations)                  |
| `split`         | Splitting variable with train, test, and validation samples |

---


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math

In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD_2024_house.csv"
house_rad = pd.read_csv(url)

# Display the first few rows of the dataset
house_rad.head()



---

# Exploratory and Graphical Analysis






## Question 1

Verify the dimensions of the dataset, the types of individual variables, and summarize the basic descriptive statistics of all variables. Plot a histogram and a density estimate for the target variable `price`. Can anything be inferred for future analysis?

In [ ]:
# Verify the dimensions of the dataset
print("\nDimensions of the dataset:", house_rad.shape)

# Data types of individual variables
print("\nData types of variables:")
print(house_rad.dtypes)

# Summarize basic descriptive statistics of all variables
print("\nDescriptive statistics:")
print(house_rad.describe(include='all'))

In [ ]:
# Plot histogram and density estimate for the target variable price
plt.figure(figsize=(12, 6))
house_rad['price'].hist(bins=100, alpha=0.7, label='Histogram', density=True)
house_rad['price'].plot(kind='kde', label='Density Estimate', color='red')
plt.title('Distribution of Sale Price')
plt.xlabel('Price')
plt.ylabel('Frequency/Density')
plt.legend()
plt.show()

The histogram suggest that the price distribution is right-skewed, with most prices concentrated in the lower range but a long tail extending to very high prices.

In [ ]:
house_rad['price'].plot.box()

Most values for price are around one milion dollars. Above four milion may be nonsensical.




### Question 2

Are all variables usable for analysis and prediction of house prices? If the data contains missing values (or strange or nonsensical observations), can they be imputed (corrected), or must they be removed from the dataset?

---


## unusable variables ##

Variables Unnamed: 0 and id may be dropped right away

In [ ]:
house_rad = house_rad.drop(columns=["Unnamed: 0", "id"])

## missing values ##

In [ ]:
# Check for missing values in each column
missing_values = house_rad.isnull().sum()

# Display columns with missing values
columns_with_nans = missing_values[missing_values > 0]

print("Columns with missing values and their counts:")
print(columns_with_nans)

no missing values

## nonsensical values ##

In [ ]:
num_plots = len(house_rad.columns)

# Define the grid size (e.g., 3 columns per row)
ncols = 4
nrows = math.ceil(num_plots / ncols)

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 5))
axes = axes.flatten()  # Flatten in case of multiple rows

# Loop through each numeric column and plot a boxplot
for ax, col in zip(axes, house_rad):
    sns.boxplot(x=house_rad[col], ax=ax)
    ax.set_title(f"Boxplot of {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.7)

# Remove any unused subplots
for ax in axes[num_plots:]:
    fig.delaxes(ax)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the grid of boxplots
plt.show()

In [ ]:
data = house_rad[
    (house_rad['bathrooms'] < 5) &
    (house_rad['sqft_living'] > 400) &
    (house_rad['sqft_living15'] > 400) &
    (house_rad['sqft_lot'] > 400) &
    (house_rad['sqft_lot15'] > 400) &
    (house_rad['grade'] < 15)
]

In [ ]:
len(data)

We remove nonensical values as the dataset has sufficient size

In [ ]:
# Plot histograms of the specified columns
data.hist(bins=30, figsize=(20, 20), grid=False, edgecolor='black', alpha=0.7)
plt.suptitle("Filtered Data", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(data, diag_kind='hist', height=2.5)
plt.show()

Zdá se, že hodnoty s vysokou cenou mohou být zatíženy chybou (panem Francem), neboť se nezdá že by reflektovaly trend a pro obrovské ceny mají velmi malou Living Area, stejně jako poměrně nízké ceny v poměru s obrovksými Living Area

---


Vzhledem k tomu, že nevíme jaké jsou správné hodnoty, navrhruji nevyhovující hodnoty vyhodit.
Např jak zmíněno výše: posledních několik hodnot u price jsou nepoměrně velké k ostatním a nedávají smysl, stejně jako hodnoty s nesmyslně velkým Living Area za neadekvátní ceny.

Proměnná grade ná zřejmě také outliera, neboť se vyskytuje jedna hodnota 232 která narušuje všechny ostatní statistiky

Proměnná Waterfall view by mohla činit problémy neboť je u většiny 0



### Question 3

For the selected variables (`price`, `sqft_living`, `grade`, `yr_built`), verify whether the split into train, test, and validation datasets was random. That is, do these variables have approximately the same distributions across the train, test, and validation groups?


In [ ]:
figsize = (12, 1.2 * len(data['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='price', y='split', inner='box', color="orange")
sns.despine(top=True, right=True, bottom=True, left=True)
plt.title('price')
plt.show()

In [ ]:

figsize = (12, 1.2 * len(data['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='sqft_living', y='split', inner='box', color='green')
sns.despine(top=True, right=True, bottom=True, left=True)
plt.title('sqft_living')
plt.show()

In [ ]:
figsize = (12, 1.2 * len(data['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='grade', y='split', inner='box', color='grey')
sns.despine(top=True, right=True, bottom=True, left=True)
plt.title('grade')
plt.show()

In [ ]:
figsize = (12, 1.2 * len(data['split'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(data, x='yr_built', y='split', inner='box', color="cyan")
sns.despine(top=True, right=True, bottom=True, left=True)
plt.title('yr_built')
plt.show()

In [ ]:
selected_variables = ['price', 'sqft_living', 'grade', 'yr_built']

# Create a 2x2 grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))  # Adjust figsize as needed
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

# Loop through each variable and corresponding subplot axis
for ax, variable in zip(axes, selected_variables):
    sns.kdeplot(
        data=data,
        x=variable,
        hue='split',
        fill=True,
        common_norm=False,
        alpha=0.5,
        ax=ax
    )
    # Set plot title and labels
    ax.set_title(f"Distribution of {variable.replace('_', ' ').title()} Across Splits", fontsize=14)
    ax.set_xlabel(variable.replace('_', ' ').title(), fontsize=12)
    ax.set_ylabel("Density", fontsize=12)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the grid of density plots
plt.show()

In [ ]:
# Group by split
train_data = data[data['split'] == 'train'].drop(columns=['split'])
test_data = data[data['split'] == 'test'].drop(columns=['split'])
validation_data = data[data['split'] == 'validation'].drop(columns=['split'])

In [ ]:
from scipy.stats import ks_2samp, f_oneway, kruskal

for variable in selected_variables:
    print(f"\n=== Analysis for {variable} ===")

    # Kolmogorov-Smirnov Test: Train vs Test
    ks_train_test = ks_2samp(train_data[variable], test_data[variable])
    print(f"KS Test (Train vs Test): Statistic={ks_train_test.statistic:.4f}, p-value={ks_train_test.pvalue:.4f}")

    # Kolmogorov-Smirnov Test: Train vs Validation
    ks_train_validation = ks_2samp(train_data[variable], validation_data[variable])
    print(f"KS Test (Train vs Validation): Statistic={ks_train_validation.statistic:.4f}, p-value={ks_train_validation.pvalue:.4f}")

    # Kolmogorov-Smirnov Test: Test vs Validation
    ks_test_validation = ks_2samp(test_data[variable], validation_data[variable])
    print(f"KS Test (Test vs Validation): Statistic={ks_test_validation.statistic:.4f}, p-value={ks_test_validation.pvalue:.4f}")

    # ANOVA
    anova_result = f_oneway(train_data[variable], test_data[variable], validation_data[variable])
    print(f"ANOVA: F-statistic={anova_result.statistic:.4f}, p-value={anova_result.pvalue:.4f}")

    # Kruskal-Wallis Test
    kruskal_result = kruskal(train_data[variable], test_data[variable], validation_data[variable])
    print(f"Kruskal-Wallis Test: H-statistic={kruskal_result.statistic:.4f}, p-value={kruskal_result.pvalue:.4f}")

Žádná z p-values není signifikantní.


# Linear Model (Use Only Training Data, i.e., split == "train")


##Question 4

Calculate the correlations between the regressors and visualize them. Also, compute the condition number (Kappa) and the variance inflation factor (VIF). If multicollinearity is present, decide which variables to use and justify your choices.

In [ ]:
correlation_matrix = train_data.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
from numpy.linalg import cond
# 2. Condition Number (Kappa)
condition_number = cond(train_data.values)
print(f"Condition Number (Kappa): {condition_number:.2f}")

# 3. Variance Inflation Factor (VIF)
# Add a constant to X for intercept
X_with_const = sm.add_constant(train_data)

vif_data = pd.DataFrame()
vif_data['Variable'] = train_data.columns
vif_data['VIF'] = [variance_inflation_factor(X_with_const.values, i + 1) for i in range(len(train_data.columns))]
print("\nVariance Inflation Factor (VIF):")
print(vif_data)

Scaler pro koeficient KAPPA

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from statsmodels.tools.tools import add_constant

# Standardize the features (but not the target variable)
scaler = StandardScaler()
new_train_standardized = scaler.fit_transform(train_data)

# Add constant (intercept) to the standardized features
X_with_const_standardized = add_constant(new_train_standardized)

# Calculate the Condition Number (Kappa)
condition_number_standardized = np.linalg.cond(X_with_const_standardized)
print(f"Condition Number (Kappa) after standardization: {condition_number_standardized}")


scaled_df = pd.DataFrame(new_train_standardized, columns=train_data.columns)

We remove columns sqft_lot15 a sqft_above

In [ ]:
cleaner_data = data.drop(columns=['sqft_lot15', 'sqft_above'])

In [ ]:
cleaner_data_num = cleaner_data.drop(columns=['split'])
correlation_matrix = cleaner_data_num.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap')
plt.show()

Scaler pro koeficient KAPPA

In [ ]:
# Standardize the features (but not the target variable)
scaler = StandardScaler()
new_train_standardized = scaler.fit_transform(cleaner_data_num)

# Add constant (intercept) to the standardized features
X_with_const_standardized = add_constant(new_train_standardized)

# Calculate the Condition Number (Kappa)
condition_number_standardized = np.linalg.cond(X_with_const_standardized)
print(f"Condition Number (Kappa) after standardization: {condition_number_standardized}")


scaled_df = pd.DataFrame(new_train_standardized, columns=cleaner_data_num.columns)

In [ ]:

# 2. Condition Number (Kappa)
condition_number = cond(cleaner_data_num.values)
print(f"Condition Number (Kappa): {condition_number:.2f}")

# 3. Variance Inflation Factor (VIF)
# Add a constant to X for intercept
X_with_const = sm.add_constant(cleaner_data_num)

vif_data = pd.DataFrame()
vif_data['Variable'] = cleaner_data_num.columns
vif_data['VIF'] = [variance_inflation_factor(X_with_const.values, i + 1) for i in range(len(cleaner_data_num.columns))]
print("\nVariance Inflation Factor (VIF):")
print(vif_data)




## Question 5

Using only the training data (split == "train") and all selected variables, find a suitable linear regression model that best predicts the target variable `price`, i.e., minimizes the mean squared error (MSE). Compare the VIF and Kappa values of the final model to those of the original regressor matrix.



In [ ]:
from itertools import combinations

def minimize_mse(data, feature_set, split, interactions=False):
    split_data = data[data['split'] == split]

    best_features = None
    lowest_mse = float("inf")
    best_model = None

    # Generate all possible subsets of features
    for k in range(1, len(feature_set) + 1):  # Try all subset sizes
        for subset in combinations(feature_set, k):
            # Fit the model with the current subset of features
            model = model_creation_smf(split_data, subset, interactions=interactions)

            # Predict on the training data
            y_hat = model.predict(split_data)
            y = split_data['price']

            # Calculate MSE manually
            mse = np.mean((y_hat - y) ** 2)

            # Update the best model if this one has a lower MSE
            if mse < lowest_mse:
                lowest_mse = mse
                best_features = subset
                best_model = model

    print(lowest_mse)

    return best_model, best_features, lowest_mse

In [ ]:
def model_creation_smf(data, selected_features, interactions=False):
    if interactions:
        formula = "price ~ " + "*".join(selected_features)
    else:
        formula = "price ~ " + "+".join(selected_features)

    model = smf.ols(formula, data=data).fit()

    return model

In [ ]:
columns = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_basement', 'yr_built',
       'yr_renovated', 'sqft_living15']

In [ ]:
feature_set = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15']
# best_model, best_features, best_mse = minimize_mse(cleaner_data, feature_set, "train", interactions=False)

('bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'sqft_living15')



 ### MSE is minimiseed by a model with all variables

In [ ]:
best_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15']
best_model =  model_creation_smf(cleaner_data[cleaner_data['split'] == "train"], best_features)

In [ ]:
print(best_model.summary())

### Logaritmická transformace

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Log-transform the target variable 'price' and define the regression formula
formula = "np.log(price + 1) ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + waterfront + view + condition + grade + sqft_basement + yr_built + yr_renovated + sqft_living15"

# Fit the linear regression model with the transformed target variable
model = smf.ols(formula=formula, data=data).fit()

# Print the summary of the regression model
print(model.summary())

best_model =  model_creation_smf(cleaner_data[cleaner_data['split'] == "train"], best_features)


Several of the variables have high values of P and low walues of coefitient - we remove: *bedrooms, sqft_living, sqft_lot, yr_renovated*

**Removal of variables**

In [ ]:
cleaner_data = cleaner_data.drop(columns=['bedrooms', 'sqft_living', 'sqft_lot', 'yr_renovated'])

In [ ]:
best_features = ['bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_basement', 'yr_built', 'sqft_living15']

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Log-transform the target variable 'price' and define the regression formula
formula = "np.log(price + 1) ~ bathrooms + floors + waterfront + view + condition + grade + sqft_basement + yr_built + sqft_living15"

# Fit the linear regression model with the transformed target variable
model = smf.ols(formula=formula, data=data).fit()

# Print the summary of the regression model
print(model.summary())

best_model =  model_creation_smf(cleaner_data[cleaner_data['split'] == "train"], best_features)

### VIF and Kappa and residuals

residuals

In [ ]:
from numpy import log

cleaned_data = cleaner_data[cleaner_data['split'] == "train"]
X = cleaned_data[best_features]
# Add constant term for the intercept in the OLS model
X_const = sm.add_constant(X)
y_l = cleaned_data['price']

# Check for zero or negative values in the target variable
print(y_l[y_l <= 0])

# Count zero or negative values
print(f"Number of zero or negative values: {(y_l <= 0).sum()}")

print(y_l.head())


# Apply log transformation to target variable
y_log = np.log(y_l + 1)  # Shift by 1 to avoid log(0)

# Re-fit the model
model_log = sm.OLS(y_log, X_const).fit()
print(model_log.summary())

# Check residuals again
predictions_log = model_log.predict(X_const)
residuals_log = y_log - predictions_log


In [ ]:
from sklearn.metrics import mean_squared_error

# Mean Squared Error
mse = mean_squared_error(y_log, predictions_log)
print(f"Mean Squared Error (MSE): {mse}")

# Residual Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions_log, y=residuals_log, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Residuals vs Predicted Values")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()

# Histogram of Residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals_log, kde=True, bins=30, color='blue')
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# QQ Plot
sm.qqplot(residuals_log, line='45', fit=True)
plt.title("QQ Plot of Residuals")
plt.show()

# Check for Heteroscedasticity
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions_log, y=abs(residuals_log), alpha=0.6)
plt.title("Absolute Residuals vs Predicted Values (Heteroscedasticity Check)")
plt.xlabel("Predicted Values")
plt.ylabel("Absolute Residuals")
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
cleaner_data_num = cleaner_data.drop(columns=['split'])

In [ ]:
# Standardize the features (but not the target variable)
scaler = StandardScaler()
new_train_standardized = scaler.fit_transform(cleaner_data_num)

# Add constant (intercept) to the standardized features
X_with_const_standardized = add_constant(new_train_standardized)

# Calculate the Condition Number (Kappa)
condition_number_standardized = np.linalg.cond(X_with_const_standardized)
print(f"Condition Number (Kappa) after standardization: {condition_number_standardized}")


scaled_df = pd.DataFrame(new_train_standardized, columns=cleaner_data_num.columns)

In [ ]:
# 3. Variance Inflation Factor (VIF)
# Add a constant to X for intercept
X_with_const = sm.add_constant(cleaner_data_num)

vif_data = pd.DataFrame()
vif_data['Variable'] = cleaner_data_num.columns
vif_data['VIF'] = [variance_inflation_factor(X_with_const.values, i + 1) for i in range(len(cleaner_data_num.columns))]
print("\nVariance Inflation Factor (VIF):")
print(vif_data)



## Question 6

For your selected model from the previous question, calculate the relevant influence measures. Provide the `id` for the top 20 observations with the highest values of DIFF, the highest leverage (hat values), and the highest Cook’s distance (i.e., 3 sets of 20 values). Which observations do you consider influential or outliers?



In [ ]:
# Assume `model` is your fitted OLS model
influence = model_log.get_influence()

# Extract influence measures
dfbetas = influence.dfbetas  # DIFF (standardized changes in coefficients)
leverage = influence.hat_matrix_diag  # Leverage (hat values)
cooks_d = influence.cooks_distance[0]  # Cook's distance

# Create a DataFrame to store these values along with observation IDs
influence_measures = pd.DataFrame({
    'Observation': range(len(dfbetas)),  # Assuming the index matches row numbers
    'Leverage': leverage,
    'Cooks_D': cooks_d,
    'DIFF': dfbetas.max(axis=1)  # Taking the largest DFBETA value for each observation
})

# Sort and select top 20 for each measure
top_diff = influence_measures.nlargest(20, 'DIFF')
top_leverage = influence_measures.nlargest(20, 'Leverage')
top_cooks_d = influence_measures.nlargest(20, 'Cooks_D')


# Display top 20 observations for each measure
print("Top 20 Observations by DIFF:")
print(top_diff)

print("\nTop 20 Observations by Leverage:")
print(top_leverage)

print("\nTop 20 Observations by Cook's Distance:")
print(top_cooks_d)



## Question 7

Validate the model graphically using residual plots (Residual vs. Fitted, QQ-plot, Cook’s distance, leverage, etc.). Based on this and the previous question, did you identify any suspicious observations in the data that might have resulted from dataset adjustments? Would you recommend removing these observations from the data?



In [ ]:
from sklearn.metrics import mean_squared_error

y = cleaner_data[cleaner_data['split']=='train']['price']
# Predictions and residuals
predictions = best_model.predict( cleaner_data[cleaner_data['split']=='train'])
residuals = y - predictions

# Mean Squared Error
mse = mean_squared_error(y, predictions)
print(f"Mean Squared Error (MSE): {mse}")

# Residual Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions, y=residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Residuals vs Predicted Values")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()

# Histogram of Residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30, color='blue')
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# QQ Plot
sm.qqplot(residuals, line='45', fit=True)
plt.title("QQ Plot of Residuals")
plt.show()

# Check for Heteroscedasticity
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions, y=abs(residuals), alpha=0.6)
plt.title("Absolute Residuals vs Predicted Values (Heteroscedasticity Check)")
plt.xlabel("Predicted Values")
plt.ylabel("Absolute Residuals")
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
def summarize_influence_measures_with_data(model, data):
    """
    Summarize influence measures, flag outliers, and include original data columns.

    :param model: Fitted regression model object from statsmodels.
    :param data: DataFrame used to fit the regression model.
    :return: DataFrame summarizing influence measures, flagged outliers, and original data.
    """
    influence = model.get_influence()

    # Extract measures
    leverage = influence.hat_matrix_diag
    cooks_distance = influence.cooks_distance[0]
    dffits = influence.dffits[0]
    dfbetas = influence.dfbetas
    cov_ratios = influence.cov_ratio

    # Number of observations and predictors
    n = int(model.nobs)
    p = int(model.df_model) + 1  # Add 1 to include the intercept

    # Rule of Thumb thresholds
    leverage_threshold = 2 * (p) / n
    cooks_distance_threshold = 4 / n
    dffits_threshold = 2 * np.sqrt((p) / n)
    dfbetas_threshold = 2 / np.sqrt(n)

    # Summarize outliers based on thresholds
    flagged = {
        'High Leverage': leverage > leverage_threshold,
        'High Cook\'s Distance': cooks_distance > cooks_distance_threshold,
        'High DFFITS': np.abs(dffits) > dffits_threshold,
    }

    # Flag observations with high DFBETAS for any predictor
    for j in range(dfbetas.shape[1]):
        flagged[f'High DFBETAS (Predictor {j})'] = np.abs(dfbetas[:, j]) > dfbetas_threshold

    # Create summary DataFrame
    summary_df = pd.DataFrame({
        'Leverage': leverage,
        'Cook\'s Distance': cooks_distance,
        'DFFITS': dffits,
        'Covariance Ratio': cov_ratios
    })

    # Add flags for rule-of-thumb violations
    for key, flag in flagged.items():
        summary_df[key] = flag

    # Combine summary DataFrame with original data
    summary_with_data = pd.concat([data.reset_index(drop=True), summary_df], axis=1)

    # Select rows where any flag is True
    flag_columns = [col for col in summary_df.columns if 'High' in col]
    flagged_observations = summary_with_data.loc[summary_df[flag_columns].any(axis=1)]

    return summary_with_data, flagged_observations


In [ ]:
# Apply the function
all_observations_with_im, flagged_observations = summarize_influence_measures_with_data(model_log, cleaned_data)

# Display the flagged observations
flagged_observations

In [ ]:
# Number of observations and predictors
n = int(model_log.nobs)
p = int(model_log.df_model) + 1  # Include intercept

# Define thresholds
leverage_threshold = 2 * p / n
cooks_d_threshold = 4 / n
dffits_threshold = 2 * np.sqrt(p / n)
dfbetas_threshold = 2 / np.sqrt(n)


summary_influence = influence.summary_frame()

In [ ]:
# Identify observations exceeding the threshold
cooks_d = summary_influence['cooks_d']
influential_obs = cooks_d > cooks_d_threshold
influential_indices = summary_influence[influential_obs].index

# Create the stem plot of Cook's Distance
plt.figure(figsize=(12, 8))
markerline, stemlines, baseline = plt.stem(
    summary_influence.index, cooks_d, markerfmt=",", basefmt=" "
)
plt.axhline(y=cooks_d_threshold, color='red', linestyle='--', label=f'Threshold ({cooks_d_threshold:.4f})')
plt.xlabel('Observation Index')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance for Each Observation")
plt.legend()

# Annotate influential observations
for idx in influential_indices:
    plt.annotate(
        str(idx),
        (idx, cooks_d[idx]),
        textcoords="offset points",
        xytext=(0, 10),  # Offset label above the point
        ha='center',
        fontsize=9,
        color='blue'
    )

plt.show()



# Robust Approach



## Question 8

If you decided to remove any observations from the dataset, work with the filtered dataset, retrain the model from Question 5, and calculate the $R^2$ statistic and MSE on both the training and test data (split == "test").

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
train_y = cleaner_data[cleaner_data['split']=='train']['price']
train_y_hat = best_model.predict(cleaner_data[cleaner_data['split']=='train'])

test_y = cleaner_data[cleaner_data['split']=='test']['price']
test_y_hat = best_model.predict(cleaner_data[cleaner_data['split']=='test'])

# Calculate R-squared
r_squared_train = r2_score(train_y, train_y_hat)
r_squared_test = r2_score(test_y, test_y_hat)


# Calculate Mean Squared Error
mse_train = mean_squared_error(train_y, train_y_hat)
mse_test = mean_squared_error(test_y, test_y_hat)

print(f"train: mse {mse_train}, R2 {r_squared_train}")
print(f"test: mse {mse_test}, R2 {r_squared_test}")




## Question 9

Estimate the regression coefficients using a robust method, such as Total Least Squares (TLS), on both the filtered dataset (after removing any observations, if applicable) and the original full dataset. Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

def total_least_squares(X, y):
    """
    Estimate regression coefficients using Total Least Squares.

    Args:
        X (numpy.ndarray): Independent variables matrix.
        y (numpy.ndarray): Dependent variable vector.

    Returns:
        numpy.ndarray: Estimated regression coefficients.
    """
    # Combine X and y into a single matrix
    A = np.hstack((X, y.reshape(-1, 1)))

    # Singular Value Decomposition
    U, S, Vt = np.linalg.svd(A, full_matrices=False)

    # Last column of V (smallest singular value direction)
    v = Vt.T[:, -1]

    # Separate coefficients and intercept
    coefs = -v[:-1] / v[-1]
    return coefs

def apply_tls(data, features, target):
    """
    Apply Total Least Squares to a dataset.

    Args:
        data (pd.DataFrame): Dataset to analyze.
        features (list): List of feature column names.
        target (str): Target column name.

    Returns:
        numpy.ndarray: Estimated regression coefficients.
    """
    X = data[features].values
    y = data[target].values
    return total_least_squares(X, y)

# Prepare data
features = best_features
target = 'price'

# TLS on filtered dataset
cleaner_data_features = cleaner_data[features]
cleaner_data_target = cleaner_data[target]
cleaner_coefficients = apply_tls(cleaner_data, features, target)

# TLS on original dataset
original_data_features = house_rad[features]
original_data_target = house_rad[target]
original_coefficients = apply_tls(house_rad, features, target)

# Predictions for performance comparison
cleaner_predictions = np.dot(cleaner_data_features, cleaner_coefficients)
original_predictions = np.dot(original_data_features, original_coefficients)

# Evaluate performance
cleaner_mse = mean_squared_error(cleaner_data_target, cleaner_predictions)
cleaner_r2 = r2_score(cleaner_data_target, cleaner_predictions)

original_mse = mean_squared_error(original_data_target, original_predictions)
original_r2 = r2_score(original_data_target, original_predictions)

# Results summary
print("Filtered Dataset TLS Coefficients:", cleaner_coefficients)
print("Original Dataset TLS Coefficients:", original_coefficients)
print("\nPerformance Comparison:")
print(f"Filtered Data - MSE: {cleaner_mse}, R2: {cleaner_r2}")
print(f"Original Data - MSE: {original_mse}, R2: {original_r2}")


In [ ]:
from numpy import log

cleaned_data = cleaner_data[cleaner_data['split'] == "train"]
X = cleaned_data[best_features]
# Add constant term for the intercept in the OLS model
X_const = sm.add_constant(X)
y_l = cleaned_data['price']

# Check for zero or negative values in the target variable
print(y_l[y_l <= 0])

# Count zero or negative values
print(f"Number of zero or negative values: {(y_l <= 0).sum()}")

print(y_l.head())


# Apply log transformation to target variable
y_log = np.log(y_l + 1)  # Shift by 1 to avoid log(0)

# Re-fit the model
model_log = sm.OLS(y_log, X_const).fit()
print(model_log.summary())

# Check residuals again
predictions_log = model_log.predict(X_const)
residuals_log = y_log - predictions_log

**Filtered Dataset:** The coefficients for the filtered dataset suggest a more stable and possibly robust model. The magnitudes of the coefficients are relatively consistent, indicating that filtering likely removed influential outliers or noisy observations.

**Original Dataset:** The coefficients for the original dataset vary significantly in magnitude. This can indicate that the presence of noisy or extreme observations influenced the model, leading to potentially less stable regression estimates.

In [ ]:
def compare_coefficients(filtered, original, feature_names):
    """
    Compare TLS coefficients from filtered and original datasets.

    Args:
        filtered (list or numpy.ndarray): Coefficients from the filtered dataset.
        original (list or numpy.ndarray): Coefficients from the original dataset.
        feature_names (list): Feature names corresponding to the coefficients.
    """
    x = range(len(feature_names))
    plt.figure(figsize=(10, 6))
    plt.bar(x, filtered, width=0.4, label='Filtered', align='center')
    plt.bar([i + 0.4 for i in x], original, width=0.4, label='Original', align='center')
    plt.xticks([i + 0.2 for i in x], feature_names, rotation=45, ha='right')
    plt.title("Comparison of Coefficients")
    plt.xlabel("Features")
    plt.ylabel("Coefficient Value")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Example usage
compare_coefficients(cleaner_coefficients, original_coefficients, best_features)



## Question 10

Explore additional robust regression approaches (e.g., Huber regression, quantile regression, or M-estimators) to estimate the regression coefficients on the filtered and full datasets. Compare the results across these methods, discussing the strengths and limitations of each approach in the context of predicting house prices in the King County area. How do these robust methods handle potential outliers or influential observations in the data?


### Huber regression

In [ ]:
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import HuberRegressor

y = cleaner_data['price']
X = add_constant(cleaner_data.drop(columns=['split']))

y_full = house_rad['price']
X_full = add_constant(house_rad.drop(columns=['split', 'sqft_living', 'sqft_above']))


# Fit Huber Regressor
huber_model = HuberRegressor()
huber_model.fit(X, y)

# Predictions and performance
y_pred_filtered = huber_model.predict(X)
mse_filtered = mean_squared_error(y, y_pred_filtered)

print("Huber Regression Coefficients (Filtered):", huber_model.coef_)
print("Huber Regression MSE (Filtered):", mse_filtered)
print()

# Refit on the full dataset
huber_model.fit(X_full, y_full)
y_pred_full = huber_model.predict(X_full)
mse_full = mean_squared_error(y_full, y_pred_full)

print("Huber Regression Coefficients (Full):", huber_model.coef_)
print("Huber Regression MSE (Full):", mse_full)

### Qualtile regression

In [ ]:
from statsmodels.regression.quantile_regression import QuantReg

# Fit Quantile Regression for the median
quantile_model = QuantReg(y, add_constant(X))
quantile_results_filtered = quantile_model.fit(q=0.5)
print("Quantile Regression Summary (Filtered):")
print(quantile_results_filtered.summary())

# Refit on the full dataset
quantile_model_full = QuantReg(y_full, add_constant(X_full))
quantile_results_full = quantile_model_full.fit(q=0.5)
print("Quantile Regression Summary (Full):")
print(quantile_results_full.summary())

### RLM

In [ ]:
from statsmodels.robust.robust_linear_model import RLM

# Fit RLM using Huber's T norm
rlm_model = RLM(y, add_constant(X))
rlm_results_filtered = rlm_model.fit()
print("RLM Summary (Filtered):")
print(rlm_results_filtered.summary())

# Refit on the full dataset
rlm_model_full = RLM(y_full, add_constant(X_full))
rlm_results_full = rlm_model_full.fit()
print("RLM Summary (Full):")
print(rlm_results_full.summary())



## Question 11

Select the final model and compare the MSE and $R^2$ on the training, test, and validation datasets. What do these values suggest about the quality of the model and potential overfitting? Is your model suitable for predicting house prices in the King County area? If so, does this prediction have any limitations?



In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
train_y = cleaner_data[cleaner_data['split']=='train']['price']
train_y_hat = best_model.predict(cleaner_data[cleaner_data['split']=='train'])

test_y = cleaner_data[cleaner_data['split']=='test']['price']
test_y_hat = best_model.predict(cleaner_data[cleaner_data['split']=='test'])

valid_y = cleaner_data[cleaner_data['split']=='validation']['price']
valid_y_hat = best_model.predict(cleaner_data[cleaner_data['split']=='validation'])

# Calculate R-squared
r_squared_train = r2_score(train_y, train_y_hat)
r_squared_test = r2_score(test_y, test_y_hat)
r_squared_valid = r2_score(valid_y, valid_y_hat)


# Calculate Mean Squared Error
mse_train = mean_squared_error(train_y, train_y_hat)
mse_test = mean_squared_error(test_y, test_y_hat)
mse_valid = mean_squared_error(valid_y, valid_y_hat)

print(f"train: mse {mse_train}, R2 {r_squared_train}")
print(f"test: mse {mse_test}, R2 {r_squared_test}")
print(f"validation: mse {mse_valid}, R2 {r_squared_valid}")

---

## Voluntary part: Machine Learning Approach


### Question 12
Apply machine learning-based linear regression methods, such as Ridge Regression, Lasso, or Elastic Net, to the dataset. Use the train-test split to evaluate model performance and focus on feature selection. Identify the most relevant features based on these methods and compare how the selected features impact the model's predictive performance. Discuss how regularization affects feature selection and the trade-offs between bias and variance in the context of house price prediction. Additionally, evaluate the stability of selected features across different methods and provide recommendations for choosing the optimal feature set.
